# Building a Stock Screener

This notebook walks you through stock screening—how to filter thousands of stocks to find investment opportunities.

## What is Stock Screening?

Imagine you're looking for a used car. You wouldn't manually check every car in the country. Instead, you'd filter:
- Price under $20,000
- Made after 2018
- Less than 50,000 miles

Stock screening is the same idea. There are ~4,000 stocks on US exchanges. Screening helps you filter to just the ones matching your criteria:
- **Value investors** might screen for low P/E ratios
- **Growth investors** might screen for high revenue growth
- **Dividend investors** might screen for high dividend yields

Let's build a screener from scratch!

In [ ]:
# Install yfinance if needed (uncomment to run)
# !pip install yfinance pandas

import yfinance as yf
import pandas as pd

# Our stock universe (normally you'd screen thousands)
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'JPM', 'BAC', 'JNJ', 'PFE']

## 1. Simplest Example: Filter by P/E Ratio

Let's start with one criterion: **P/E ratio < 20**

The P/E (Price-to-Earnings) ratio tells you how much you're paying for each dollar of profit:
- P/E = 10 means you pay $10 for every $1 of annual earnings
- Lower P/E often means "cheaper" (but not always better!)

In [ ]:
# Get stock data
stocks_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    # Extract key metrics
    pe_ratio = info.get('trailingPE', None)
    
    if pe_ratio:  # Only include if P/E exists
        stocks_data.append({
            'Ticker': ticker,
            'Company': info.get('shortName', ticker),
            'P/E': round(pe_ratio, 2)
        })

# Create DataFrame
df = pd.DataFrame(stocks_data)
print("All stocks:")
print(df)
print()

# Apply our screen: P/E < 20
screened = df[df['P/E'] < 20]
print("Stocks with P/E < 20:")
print(screened)

### What Happened?

1. We downloaded data for 10 stocks using `yfinance`
2. We extracted the P/E ratio for each stock
3. We filtered: `df[df['P/E'] < 20]` kept only stocks where P/E < 20
4. The universe **narrowed** from 10 stocks to just a few

This is the core of screening: **start with many, filter to few**.

### 🧪 Experiment: Change the Threshold

Try screening for P/E < 15 or P/E < 30. How does the number of results change?

In [ ]:
# Your turn: Try different P/E thresholds
# screened = df[df['P/E'] < YOUR_NUMBER]


## 2. Adding a Second Filter: Profitability (ROE)

P/E alone doesn't tell the whole story. A stock might be cheap because the company is struggling!

Let's add **ROE (Return on Equity) > 15%** to find profitable companies:
- ROE = how much profit a company generates from shareholder equity
- ROE = 15% means the company earns 15 cents of profit for every $1 of equity

Now we're screening for: **Value + Quality**

In [ ]:
# Rebuild with ROE included
stocks_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    pe_ratio = info.get('trailingPE', None)
    roe = info.get('returnOnEquity', None)
    
    if pe_ratio and roe:
        stocks_data.append({
            'Ticker': ticker,
            'Company': info.get('shortName', ticker),
            'P/E': round(pe_ratio, 2),
            'ROE (%)': round(roe * 100, 2)  # Convert to percentage
        })

df = pd.DataFrame(stocks_data)
print("All stocks:")
print(df)
print()

# Two-factor screen: Low P/E AND High ROE
screened = df[(df['P/E'] < 20) & (df['ROE (%)'] > 15)]
print("Stocks with P/E < 20 AND ROE > 15%:")
print(screened)

### What Happened?

1. We added a second metric: ROE (profitability)
2. We used **two conditions** with `&` (AND): `(df['P/E'] < 20) & (df['ROE (%)'] > 15)`
3. The list got even smaller—only stocks that pass BOTH tests

**Key insight**: More filters = fewer results, but hopefully higher quality.

### 🧪 Experiment: Try OR Instead of AND

What if you want stocks that are EITHER cheap OR profitable (not necessarily both)?

Replace `&` with `|` (OR) and see what happens.

In [ ]:
# Your turn: Try using OR instead of AND
# screened = df[(df['P/E'] < 20) | (df['ROE (%)'] > 15)]


## 3. Build Up: Multi-Factor Screen (Value + Quality + Momentum)

Professional quant funds often combine multiple factors:
- **Value**: Is it cheap? (Low P/E)
- **Quality**: Is it profitable? (High ROE)
- **Momentum**: Is it going up? (52-week performance)

Let's build a 3-factor screener:

In [ ]:
# Full multi-factor screen
stocks_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    # Get historical data for momentum calculation
    hist = stock.history(period='1y')
    
    pe_ratio = info.get('trailingPE', None)
    roe = info.get('returnOnEquity', None)
    
    if pe_ratio and roe and len(hist) > 0:
        # Calculate 52-week return
        year_return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
        
        stocks_data.append({
            'Ticker': ticker,
            'Company': info.get('shortName', ticker),
            'P/E': round(pe_ratio, 2),
            'ROE (%)': round(roe * 100, 2),
            '1Y Return (%)': round(year_return, 2)
        })

df = pd.DataFrame(stocks_data)
print("All stocks with metrics:")
print(df)
print()

# Three-factor screen
screened = df[
    (df['P/E'] < 25) &           # Value: Not too expensive
    (df['ROE (%)'] > 15) &       # Quality: Profitable
    (df['1Y Return (%)'] > 10)   # Momentum: Positive trend
]

print("Stocks passing all 3 factors:")
print(screened)

### What Happened?

1. We calculated **52-week return** as a momentum indicator
2. We combined THREE factors with `&` (all must be true)
3. We get a highly filtered list of stocks that are:
   - Reasonably priced (P/E < 25)
   - Profitable (ROE > 15%)
   - Trending upward (1Y return > 10%)

This is similar to how quant hedge funds build factor models!

### 🧪 Experiment: Rank Instead of Filter

Instead of hard cutoffs, you could **rank** stocks by a combined score.

Try creating a simple score: `Score = ROE - P/E + 1Y Return`

In [ ]:
# Your turn: Create a scoring system
df['Score'] = df['ROE (%)'] - df['P/E'] + df['1Y Return (%)']
top_scored = df.sort_values('Score', ascending=False).head(5)
print("Top 5 stocks by score:")
print(top_scored[['Ticker', 'Company', 'Score']])

## 4. Connect to Real-World Strategies

### How Quant Funds Use Screening

1. **AQR Capital** (~$100B hedge fund):
   - Screens thousands of stocks across global markets
   - Uses value, momentum, quality, and low-volatility factors
   - Rebalances monthly based on updated screens

2. **Renaissance Technologies** (famous quant fund):
   - Uses hundreds of factors (not just 3!)
   - Combines statistical signals, machine learning, and market microstructure
   - Screens update in real-time

3. **Your Personal Use**:
   - Start with broad screen (e.g., P/E < 20)
   - Add quality filters (ROE, profit margins, debt levels)
   - Manually research the ~10-20 stocks that pass
   - Make informed investment decisions

### Common Screening Strategies

| Strategy | Filters |
|----------|--------|
| **Value** | Low P/E, Low P/B, High dividend yield |
| **Growth** | High revenue growth, High earnings growth, Positive earnings surprises |
| **Quality** | High ROE, High profit margin, Low debt |
| **Momentum** | 6-month return > 0, 52-week high proximity, Volume surge |
| **Dividend** | Dividend yield > 3%, Dividend growth > 5%, Payout ratio < 60% |

## 🏋️ Practice Exercise

Build a screen for **"Undervalued Growth Stocks"**

These are companies that are:
1. Growing revenue quickly
2. Not priced too expensively
3. Financially healthy

Your task:
- Filter for **P/E < 30** (not too expensive for a growth stock)
- Filter for **Revenue Growth > 10%** (year-over-year)
- Filter for **Current Ratio > 1.5** (healthy balance sheet)

Try to build this screen yourself before looking at the solution!

In [ ]:
# Your turn: Build an "undervalued growth" screener

stocks_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    # Extract metrics you need:
    # - P/E ratio: info.get('trailingPE')
    # - Revenue growth: info.get('revenueGrowth') 
    # - Current ratio: info.get('currentRatio')
    
    # Your code here:
    pass

# Create DataFrame and apply filters
# df = pd.DataFrame(stocks_data)
# screened = df[...your conditions...]


## 💡 Solution (Try yourself first!)

In [ ]:
# Solution: Undervalued Growth Screen

stocks_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info
    
    pe_ratio = info.get('trailingPE', None)
    revenue_growth = info.get('revenueGrowth', None)
    current_ratio = info.get('currentRatio', None)
    
    if pe_ratio and revenue_growth and current_ratio:
        stocks_data.append({
            'Ticker': ticker,
            'Company': info.get('shortName', ticker),
            'P/E': round(pe_ratio, 2),
            'Revenue Growth (%)': round(revenue_growth * 100, 2),
            'Current Ratio': round(current_ratio, 2)
        })

df = pd.DataFrame(stocks_data)
print("All stocks:")
print(df)
print()

# Apply undervalued growth criteria
screened = df[
    (df['P/E'] < 30) &                    # Not too expensive
    (df['Revenue Growth (%)'] > 10) &     # Strong growth
    (df['Current Ratio'] > 1.5)           # Financially healthy
]

print("Undervalued Growth Stocks:")
print(screened)
print()
print(f"Found {len(screened)} stocks out of {len(df)} that meet all criteria.")

### What the Solution Does

1. **P/E < 30**: Filters out extremely expensive stocks (tech stocks often have P/E > 50)
2. **Revenue Growth > 10%**: Ensures the company is actually growing
3. **Current Ratio > 1.5**: Checks that current assets > current liabilities (can pay short-term debts)

**Current Ratio explained**:
- Current Ratio = Current Assets / Current Liabilities
- Ratio = 1.5 means $1.50 in assets for every $1.00 in short-term debt
- Higher = safer financial position

This combination gives you companies that are:
- Growing (revenue up)
- Reasonably priced (P/E not crazy)
- Not at risk of bankruptcy (healthy balance sheet)

## Key Takeaways

1. **Screening = Filtering**: Start with thousands of stocks, narrow to a few dozen
2. **Combine Factors**: Use multiple criteria (value + quality + momentum) for better results
3. **AND vs OR**: `&` requires all conditions, `|` requires at least one
4. **No Magic Formula**: Different strategies work in different market conditions
5. **Starting Point**: Screens don't tell you what to buy—they tell you what to research

### Next Steps

1. Try screening with different factors:
   - Dividend yield (`info.get('dividendYield')`)
   - Profit margin (`info.get('profitMargins')`)
   - Debt-to-equity (`info.get('debtToEquity')`)

2. Screen a larger universe (S&P 500 tickers)

3. Backtest your screen: Would stocks that passed your screen 1 year ago have outperformed?

4. Combine with fundamental analysis: Use screening to find candidates, then manually research them

### Resources

- **Finviz.com**: Free stock screener with pre-built filters
- **Yahoo Finance**: Has a built-in screener tool
- **QuantConnect**: Platform for backtesting quantitative strategies
- **"Quantitative Value" by Wesley Gray**: Book on factor-based screening

Happy screening! 📊